In [29]:
import popy.utils as utils
import popy
from popy.pop_maker import PopMaker
import pandas as pd

# Static networks with Popy

Assume you are an epidemiologist and want to examine a virus's propagation within schools using simulation models.
As viruses are transmitted via contacts between people, which can be represented as network graphs, network data on the contacts within a school would be a perfect base for building the simulation.
However, in most situations, network data is not available.
In those cases, Popy can be a solution for missing network data.
Popy enables you to create artificial networks by defining different locations where specific people meet.

In the following, we want to recreate the contact network of a school.
Assume we have access to the following sample of individual data collected in a school.
We use this data as a starting point for our network.

In [30]:
df_school = pd.read_csv("example_school_data.csv", sep=";")
df_school.head()

,status,gender,grade,hours,friend_group
0,pupil,m,1.0,4,1
1,pupil,w,1.0,4,1
2,pupil,m,1.0,4,2
3,pupil,w,1.0,4,2
4,pupil,m,2.0,4,3


In [31]:
df_school.tail()

,status,gender,grade,hours,friend_group
37,teacher,w,NaN,9,0
38,teacher,m,NaN,10,0
39,teacher,w,NaN,5,0
40,social_worker,w,NaN,6,0
41,social_worker,w,NaN,4,0


## The first location

Lets start with the definition of our first location: a class room.

In [32]:
class ClassRoom(popy.Location):
    pass

Using the `PopMaker`, we can create both the population of agents and the location objects.
If we are only interested in the network, only the agents are relevant to us, as they contain all the relevant network information.
If we want to build a simulation model, the locations become relevant, too.
For now, we only focus on creating static networks and, thus, ignore the created locations.

In [33]:
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

In [34]:
locations

LocationList (21 objects)

One of best ways to explore and validate the resulting network is to visualize it:

In [35]:
utils.plot_network(agents=agents, node_attrs=df_school.columns)

The network graph shows that if we do not specify the location further, one location of this type is created for each pair of random agents.

## Setting the size

In the next step, we specify the number of people in one classroom.
In this example, we assume tiny classrooms of four agents.

In [36]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4

pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns)

As you can see, some classrooms have more than four members.
If you want to make sure that no classroom has more than four members, set the location attribute `allow_overcrowding` to `False`:

In [37]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
        self.allow_overcrowding = False

pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns)

## Specifying location visitors

The classrooms above are made of all agents, even social workers.
Initially, we only want pupils to be in the classrooms (we add teachers to classrooms later).
To realize that, we use the location method `filter()`:

In [38]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"

pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

## Building separated locations

If we color the nodes by the grade of the agents, we can see that the classrooms are not separated by grade.

In [39]:
utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

To ensure that classes are separated by grade, we use the location method `split()` to implement separated classrooms for each grade.

In [40]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade


pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

## Keeping agents together

The following plot shows that the members of one friend group are distributed over different classrooms.

In [41]:
utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="friend_group")

Although this is a very realistic situation, in this example, we assume that all friend group members are always in the same class.
To implement that, we use the location method `stick_together()`:

In [42]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="friend_group")

## Edge weights

Until now, all edges between nodes have a weight of `1`.
The location method `weight()` can be used to set different weights.
In the following, we set the weight of all edges generated by a classroom to `5`.

In [43]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return 5

pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

To implement agent-specific weights, we let `weight()` return the agent attribute `agent.hours`:

In [44]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    

pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

As the value returned by `location.weight()` refers to the weight between the agent and the location, all the weights between the agents and the locations must be somehow combined when determining the weight between agents (aka graph projection).
The location method `project_weights()` defines how those weights are combined.
By default, `project_weights()` returns the smallest weight of the two to be combined.
The code cell below shows how `project_weights()` combines the two weights by default.
In this example, we keep this way of combining the weights, but this method could be easily rewritten.

In [45]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        n_agents = len(self.agents)
        return agent.status == "pupil" and n_agents <= 2
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

## More than one location

In the next step, we introduce a school as a second type of location.
Each school has about `20` members.

In [46]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

class School(popy.Location):
    def setup(self):
        self.size = 20

pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom, School])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

## Nesting locations

The plot above shows two big clusters.
Each of those clusters represents one school.
The plot above also shows something unrealistic: The schools are connected because members of one class are not always in the same school.
We can use the `School`-method `stick_together()` to fix this issue.
This works because each agent gets a new attribute named after the location class and is set to a location identifier when assigned to a location.
This attribute can be used afterward to define the structure of other locations.

In [47]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
       return min([self.get_weight(agent1), self.get_weight(agent2)])

class School(popy.Location):
    def setup(self):
        self.size = 22

    def stick_together(self, agent):
        return agent.ClassRoom


pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom, School])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

However, the agents must be assigned to the classrooms before getting assigned to the schools.
If we build schools before classrooms, the above method of sticking together classrooms does not work anymore:

In [48]:
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[School, ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="grade")

As we saw, the method `stick_together()` can be used to nest locations into other locations.
However, this approach is limited because we can only specify one location class as the return value of `stick_together()`.

Another way to nest locations into other locations is to use the location method `nest()`.
The method `nest()` can return `None` or a location class.
If `nest()` returns a location class, the location is nested into the returned location class.
For instance, to nest classrooms within schools, we must define the method `nest()` for the location `ClassRoom` and let this method return `School`.
Again, the order of location creation plays a crucial role: The level-1 location must always be created after the level-2 location.

In [49]:
class ClassRoom(popy.Location):
    def setup(self):
        self.size = 4
    
    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])
    
    def nest(self):
        return School

class School(popy.Location):
    def setup(self):
        self.size = 20


pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[School, ClassRoom])

utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

`nest()` allows us to nest as many locations in as many levels as we want.
However, `nest()` has one disadvantage ...

## Bringing together different agents

In [50]:
class TeachersInClassRoom(popy.Location):
    def setup(self):
        self.size = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.Location):
    def setup(self):
        self.size = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade

class ClassRoom(popy.Location):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]
    
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom])
utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

In [51]:
class TeachersInClassRoom(popy.Location):
    def setup(self):
        self.size = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.Location):
    def setup(self):
        self.size = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group

class ClassRoom(popy.Location):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])
    
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom, School])
utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

In [52]:
class TeachersInClassRoom(popy.Location):
    def setup(self):
        self.size = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.Location):
    def setup(self):
        self.size = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group

class ClassRoom(popy.Location):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

class School(popy.Location):
    def setup(self):
        self.size = 20
    
    def stick_together(self, agent):
        return agent.ClassRoom
    
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[ClassRoom, School])
utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")

In [53]:
class TeachersInClassRoom(popy.Location):
    def setup(self):
        self.size = 1

    def filter(self, agent):
        return agent.status == "teacher"

class PupilsInClassRoom(popy.Location):
    def setup(self):
        self.size = 4

    def filter(self, agent):
        return agent.status == "pupil"
    
    def split(self, agent):
        return agent.grade
    
    def stick_together(self, agent):
        return agent.friend_group

class ClassRoom(popy.Location):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]
    
    def weight(self, agent):
        return agent.hours
    
    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])
    
    def nest(self):
        return School

class School(popy.Location):
    def setup(self):
        self.size = 20
    
pop_maker = PopMaker()
agents, locations = pop_maker.make(df=df_school, location_classes=[School, ClassRoom])
utils.plot_network(agents=agents, node_attrs=df_school.columns, node_color="status")